In [1]:
import pandas as pd
patients = pd.read_csv("Patient Matching Data.csv")

In [2]:
patients.head()

,GroupID,PatientID,Patient Acct #,First Name,MI,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,...,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,1,1,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,1,2,NaN,Suttin,James,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,...,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,1,3,247028705-7,Sutton,J,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,...,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,NaN,Sutton,NaN,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,NaN,SUTTON,NaN,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,...,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
cleanPatients = patients.drop(["GroupID", "MI"], axis =1)
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,Suttin,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Sutton,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SUTTON,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def validateAcc(acc):
    if len(str(acc)) != 11:
        return (float('NaN'))
    else:
        if acc[9] != "-":
            return (float('NaN'))
        else:
            return acc

cleanPatients["Patient Acct #"] = cleanPatients["Patient Acct #"].apply(validateAcc)


In [5]:
cleanPatients.head(30)

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,Suttin,Power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,Sutton,Power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Sutton,Power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,SUTTON,POWER,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Sutton,Powers,12/15/2000,Female,1619 Maple Lane,NaN,Fresno,California,93726.0,Sutton,NaN,Lore,NaN,NaN,NaN,NaN,NaN
6,NaN,Sutton,Powers,12/15/2000,Female,1619 Maple Lane,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,757393348-7,Mikel,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,Arizona,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0
8,757393348-7,Mikel,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,Arizona,85219.0,NaN,NaN,NaN,81089 Kenwood Park,NaN,Chattanooga,Tennessee,37416.0
9,757393348-7,Michael,Trayton,9/20/2000,Male,715 Jana Point,NaN,Apache Junction,AZ,85219.0,NaN,NaN,NaN,81089 Kenwood Park,7,Chattanooga,Tennessee,37416.0


In [6]:
tokenized_name = cleanPatients['First Name'].apply(lambda x: x.split())

tokenized_name.head()

0    [Sutton]
1    [Suttin]
2    [Sutton]
3    [Sutton]
4    [SUTTON]
Name: First Name, dtype: object

In [7]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_name = tokenized_name.apply(lambda x: [ps.stem(i) for i in x])

tokenized_name.head()

0    [sutton]
1    [suttin]
2    [sutton]
3    [sutton]
4    [sutton]
Name: First Name, dtype: object

In [8]:
tokenized_name.head(40)

0      [sutton]
1      [suttin]
2      [sutton]
3      [sutton]
4      [sutton]
5      [sutton]
6      [sutton]
7       [mikel]
8       [mikel]
9     [michael]
10       [mike]
11      [mikel]
12      [mikel]
13      [dalia]
14      [dalia]
15     [myrvyn]
16     [mervin]
17     [myrvin]
18     [mervyn]
19     [mervin]
20      [cammi]
21        [cam]
22    [cameron]
23     [elenor]
24     [elenor]
25    [elenora]
26      [janet]
27      [janet]
28      [darbi]
29      [darbi]
30    [maynard]
31    [maynard]
32    [maynard]
33    [maynard]
34       [dant]
35       [dant]
36       [rena]
37    [annetta]
38        [ann]
39     [annett]
Name: First Name, dtype: object

In [9]:
 def lowerstr(string):
        if type(string) != float:
            return(string.lower())
    
cleanPatients["First Name"] = cleanPatients['First Name'].apply(lowerstr)
cleanPatients["Last Name"] = cleanPatients['Last Name'].apply(lowerstr)

cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code
0,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,New York,10557.0
1,NaN,suttin,power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,California,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,New York,10557.0
2,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,sutton,power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,sutton,power,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,California,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [11]:
def convert_state(state):
    if state in us_state_abbrev:
        return us_state_abbrev[state]
    
    return state

cleanPatients["Current State"] = cleanPatients["Current State"].apply(convert_state)
cleanPatients["Previous State"] = cleanPatients["Previous State"].apply(convert_state)

In [12]:
cleanPatients["Current State"].value_counts()

CA      35
TX      21
AZ      21
WA      13
NM       9
UT       9
SC       8
HI       6
PA       6
NC       6
OH       6
NY       5
WI       5
MA       5
MD       5
MI       4
KY       4
VA       4
OK       3
GA       3
OR       3
NE       3
CO       2
ME       2
CT       1
Main     1
MN       1
FL       1
PR       1
Name: Current State, dtype: int64

In [13]:
cleanPatients["Previous State"].value_counts()

TX    13
OH    10
CA     9
OK     8
AZ     6
NY     5
GA     5
FL     5
IA     4
AL     3
TN     3
PA     2
SC     2
MN     2
HI     1
IL     1
KY     1
CT     1
NC     1
Name: Previous State, dtype: int64

In [14]:
# Maps letters to phonetic groups
soundex_map = { 
    **dict.fromkeys("BFPV", '1'),
    **dict.fromkeys("CGJKQSXZ", '2'),
    **dict.fromkeys("DT", '3'),
    **dict.fromkeys("L", '4'),
    **dict.fromkeys("MN", '5'),
    **dict.fromkeys("R", '6'),
    **dict.fromkeys("AEIOUHWY", '.'),
}

soundex_map["J"]

'2'

In [15]:
def get_soundex(token):
    """Get the soundex code for the string"""
    token = token.upper()

    # first letter of input is always the first letter of soundex
    soundex = token[0]
    
    for char in token[1:]:
            code = soundex_map[char]
            if code != soundex[-1]:
                soundex += code

    # remove vowels and 'H', 'W' and 'Y' from soundex
    soundex = soundex.replace(".", "")
    
    # trim or pad to make soundex a 4-character code
    soundex = soundex[:4].ljust(4, "0")
        
    return soundex

get_soundex("Maine")

'M500'

In [16]:
print(get_soundex("Mervin"))
print(get_soundex("Mikel"))
print(get_soundex("Mikel"))
print(get_soundex("Michael"))
print(get_soundex("Mike"))




M615
M240
M240
M240
M200


In [17]:
soundex_map["Z"]

'2'

In [18]:

from metaphone import doublemetaphone
from enum import Enum

class Threshold(Enum):
    WEAK = 0
    NORMAL = 1
    STRONG = 2

def double_metaphone(value):
    print(doublemetaphone(value))
    return doublemetaphone(value)

#(Primary Key = Primary Key) = Strongest Match
#(Secondary Key = Primary Key) = Normal Match
#(Primary Key = Secondary Key) = Normal Match
#(Alternate Key = Alternate Key) = Minimal Match
def double_metaphone_compare(tuple1,tuple2,threshold):
    if threshold == Threshold.WEAK:
        if tuple1[1] == tuple2[1]:
            return True
    elif threshold == Threshold.NORMAL:
        if tuple1[0] == tuple2[1] or tuple1[1] == tuple2[0]:
            return True
    else:
        if tuple1[0] == tuple2[0]:
            return True
    return False

In [19]:
def getMetaphone(string):
    met = double_metaphone(string)
    return met[0]
cleanPatients["metFirst"] = cleanPatients["First Name"].apply(getMetaphone)

('STN', '')
('STN', '')
('STN', '')
('STN', '')
('STN', '')
('STN', '')
('STN', '')
('MKL', '')
('MKL', '')
('MKL', 'MXL')
('MK', '')
('MKL', '')
('MKL', '')
('TL', '')
('TL', '')
('MRFN', '')
('MRFN', '')
('MRFN', '')
('MRFN', '')
('MRFN', '')
('KM', '')
('KM', '')
('KMRN', '')
('ALNR', '')
('ALNR', '')
('ALNR', '')
('JNT', 'ANT')
('JNT', 'ANT')
('TRP', '')
('TRP', '')
('MNRT', '')
('MNRT', '')
('MNRT', '')
('MNRT', '')
('TNT', '')
('TNT', '')
('RN', '')
('ANT', '')
('AN', '')
('ANT', '')
('ANT', '')
('RPRS', '')
('RPRT', '')
('RPRS', '')
('RPRS', '')
('TPN', '')
('TP', '')
('TMN', '')
('TMN', '')
('TMN', '')
('TMN', '')
('FRNK', 'FRNKS')
('FRNK', '')
('FRNK', 'FRNKS')
('FRNK', 'FRNKS')
('FRNK', 'FRNKS')
('TRK', '')
('TRK', '')
('TRK', '')
('TRK', '')
('TRK', '')
('PTN', '')
('PTN', '')
('PTN', '')
('PTN', '')
('PTN', '')
('PTN', '')
('PTN', '')
('PTN', '')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('K0RN', 'KTRN')
('KTRN', '

In [20]:
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code,metFirst
0,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,NY,10557.0,STN
1,NaN,suttin,power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,CA,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,NY,10557.0,STN
2,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN
3,NaN,sutton,power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN
4,NaN,sutton,power,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN


In [21]:
from fuzzywuzzy import fuzz
#fuzz.ratio('Deluxe Room, 1 King Bed', 'Deluxe King Room')
#partial_ratio 
#token_sort_ratio


In [22]:
fuzz.partial_ratio(cleanPatients["First Name"][0], cleanPatients["First Name"][1])

83

In [23]:
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,Previous First Name,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code,metFirst
0,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,NY,10557.0,STN
1,NaN,suttin,power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,CA,93726.0,NaN,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,NY,10557.0,STN
2,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN
3,NaN,sutton,power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN
4,NaN,sutton,power,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,CA,93726.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN


In [24]:
cleanPatients["names"] = cleanPatients['First Name'].fillna('') + cleanPatients['Last Name'].fillna('')

In [28]:
cleanPatients["address"] = cleanPatients['Current Street 1'].fillna('') + \
    cleanPatients['Current City'].fillna('') + cleanPatients['Current State'].fillna('')
cleanPatients["address"] = cleanPatients["address"].apply(lowerstr)
cleanPatients.head()

,Patient Acct #,First Name,Last Name,Date of Birth,Sex,Current Street 1,Current Street 2,Current City,Current State,Current Zip Code,...,Previous MI,Previous Last Name,Previous Street 1,Previous Street 2,Previous City,Previous State,Previous Zip Code,metFirst,names,address
0,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,...,NaN,NaN,2 Erie Crossing,Apt 9,Mount Vernon,NY,10557.0,STN,suttonpower,1858 sullivan parkwayfresnoca
1,NaN,suttin,power,9/21/1945,Male,1859 Sullivan Parkway,#2,Fresno,CA,93726.0,...,NaN,NaN,2 Erie Crossing,Apartment # 9,Mount Vernon,NY,10557.0,STN,suttinpower,1859 sullivan parkwayfresnoca
2,247028705-7,sutton,power,9/20/1945,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN,suttonpower,1858 sullivan parkwayfresnoca
3,NaN,sutton,power,9/20/1954,Male,1858 Sullivan Parkway,NaN,Fresno,CA,93726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN,suttonpower,1858 sullivan parkwayfresnoca
4,NaN,sutton,power,9/20/1954,Male,1858 SULLIVAN PKWAY,APT 2,FRESNO,CA,93726.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,STN,suttonpower,1858 sullivan pkwayfresnoca


In [63]:
cleanPatients["assigned"] = 0
cleanPatients["groupno"] = -1
groups = []
groups.append([cleanPatients["names"][0]])
print(groups)
for index, row in cleanPatients.iterrows():
    if row['assigned'] == 1:
        continue
    else:
        for groupIndex, x in enumerate(groups):
            temp = fuzz.ratio(x[0],row["names"])
            print(f'fuzz for {x[0]} and {row["names"]} = {temp},  index is {groupIndex}')
            if(temp > 80 ):
                cleanPatients.loc[index, "groupno"] = index
                groups[index].append(row["names"])
                break
        else:
            groups.append([row["names"]])
            
         

     

[['suttonpower']]
fuzz for suttonpower and suttonpower = 100,  index is 0
fuzz for suttonpower and suttinpower = 91,  index is 0
fuzz for suttonpower and suttonpower = 100,  index is 0
fuzz for suttonpower and suttonpower = 100,  index is 0
fuzz for suttonpower and suttonpower = 100,  index is 0
fuzz for suttonpower and suttonpowers = 96,  index is 0
fuzz for suttonpower and suttonpowers = 96,  index is 0
fuzz for suttonpower and mikeltrayton = 35,  index is 0
fuzz for suttonpower and mikeltrayton = 35,  index is 0
fuzz for mikeltrayton and mikeltrayton = 100,  index is 1
fuzz for suttonpower and michaeltrayton = 32,  index is 0
fuzz for mikeltrayton and michaeltrayton = 85,  index is 1
fuzz for suttonpower and miketrayton = 36,  index is 0
fuzz for mikeltrayton and miketrayton = 96,  index is 1
fuzz for suttonpower and mikeltrayton = 35,  index is 0
fuzz for mikeltrayton and mikeltrayton = 100,  index is 1
fuzz for suttonpower and mikeltrayton = 35,  index is 0
fuzz for mikeltrayton a

fuzz for cameronwindles and betteannepurten = 28,  index is 5
fuzz for elenoremacdonnell and betteannepurten = 38,  index is 6
fuzz for janetbiasini and betteannepurten = 37,  index is 7
fuzz for darbysteinson and betteannepurten = 36,  index is 8
fuzz for maynardskirvan and betteannepurten = 28,  index is 9
fuzz for dantewillder and betteannepurten = 30,  index is 10
fuzz for renahawkshaw and betteannepurten = 22,  index is 11
fuzz for annettadelamere and betteannepurten = 53,  index is 12
fuzz for annettedelmar and betteannepurten = 43,  index is 13
fuzz for robersgammade and betteannepurten = 29,  index is 14
fuzz for tobinrutigliano and betteannepurten = 33,  index is 15
fuzz for damonsteer and betteannepurten = 32,  index is 16
fuzz for francoismandifield and betteannepurten = 24,  index is 17
fuzz for drackecoslett and betteannepurten = 21,  index is 18
fuzz for betteannepurton and betteannepurten = 93,  index is 19
fuzz for suttonpower and betteanepurton = 40,  index is 0
fuzz f

fuzz for elenoremacdonnell and kimberlaylygoe = 32,  index is 6
fuzz for janetbiasini and kimberlaylygoe = 15,  index is 7
fuzz for darbysteinson and kimberlaylygoe = 22,  index is 8
fuzz for maynardskirvan and kimberlaylygoe = 29,  index is 9
fuzz for dantewillder and kimberlaylygoe = 31,  index is 10
fuzz for renahawkshaw and kimberlaylygoe = 15,  index is 11
fuzz for annettadelamere and kimberlaylygoe = 28,  index is 12
fuzz for annettedelmar and kimberlaylygoe = 22,  index is 13
fuzz for robersgammade and kimberlaylygoe = 37,  index is 14
fuzz for tobinrutigliano and kimberlaylygoe = 34,  index is 15
fuzz for damonsteer and kimberlaylygoe = 25,  index is 16
fuzz for francoismandifield and kimberlaylygoe = 25,  index is 17
fuzz for drackecoslett and kimberlaylygoe = 30,  index is 18
fuzz for betteannepurton and kimberlaylygoe = 28,  index is 19
fuzz for katharinastopper and kimberlaylygoe = 33,  index is 20
fuzz for kimberleylygoe and kimberlaylygoe = 93,  index is 21
fuzz for sutto

fuzz for suttonpower and karlottamacgray = 23,  index is 0
fuzz for mikeltrayton and karlottamacgray = 44,  index is 1
fuzz for daliabibb and karlottamacgray = 25,  index is 2
fuzz for myrvynbuddock and karlottamacgray = 21,  index is 3
fuzz for cammywindless and karlottamacgray = 21,  index is 4
fuzz for cameronwindles and karlottamacgray = 21,  index is 5
fuzz for elenoremacdonnell and karlottamacgray = 31,  index is 6
fuzz for janetbiasini and karlottamacgray = 22,  index is 7
fuzz for darbysteinson and karlottamacgray = 21,  index is 8
fuzz for maynardskirvan and karlottamacgray = 28,  index is 9
fuzz for dantewillder and karlottamacgray = 22,  index is 10
fuzz for renahawkshaw and karlottamacgray = 30,  index is 11
fuzz for annettadelamere and karlottamacgray = 40,  index is 12
fuzz for annettedelmar and karlottamacgray = 43,  index is 13
fuzz for robersgammade and karlottamacgray = 36,  index is 14
fuzz for tobinrutigliano and karlottamacgray = 27,  index is 15
fuzz for damonstee

fuzz for mikeltrayton and paulifairclough = 22,  index is 1
fuzz for daliabibb and paulifairclough = 42,  index is 2
fuzz for myrvynbuddock and paulifairclough = 14,  index is 3
fuzz for cammywindless and paulifairclough = 21,  index is 4
fuzz for cameronwindles and paulifairclough = 21,  index is 5
fuzz for elenoremacdonnell and paulifairclough = 25,  index is 6
fuzz for janetbiasini and paulifairclough = 30,  index is 7
fuzz for darbysteinson and paulifairclough = 21,  index is 8
fuzz for maynardskirvan and paulifairclough = 28,  index is 9
fuzz for dantewillder and paulifairclough = 22,  index is 10
fuzz for renahawkshaw and paulifairclough = 22,  index is 11
fuzz for annettadelamere and paulifairclough = 27,  index is 12
fuzz for annettedelmar and paulifairclough = 29,  index is 13
fuzz for robersgammade and paulifairclough = 21,  index is 14
fuzz for tobinrutigliano and paulifairclough = 33,  index is 15
fuzz for damonsteer and paulifairclough = 16,  index is 16
fuzz for francoism

fuzz for myrvynbuddock and tamiebadman = 25,  index is 3
fuzz for cammywindless and tamiebadman = 33,  index is 4
fuzz for cameronwindles and tamiebadman = 32,  index is 5
fuzz for elenoremacdonnell and tamiebadman = 29,  index is 6
fuzz for janetbiasini and tamiebadman = 43,  index is 7
fuzz for darbysteinson and tamiebadman = 25,  index is 8
fuzz for maynardskirvan and tamiebadman = 40,  index is 9
fuzz for dantewillder and tamiebadman = 26,  index is 10
fuzz for renahawkshaw and tamiebadman = 26,  index is 11
fuzz for annettadelamere and tamiebadman = 38,  index is 12
fuzz for annettedelmar and tamiebadman = 42,  index is 13
fuzz for robersgammade and tamiebadman = 33,  index is 14
fuzz for tobinrutigliano and tamiebadman = 31,  index is 15
fuzz for damonsteer and tamiebadman = 29,  index is 16
fuzz for francoismandifield and tamiebadman = 34,  index is 17
fuzz for drackecoslett and tamiebadman = 17,  index is 18
fuzz for betteannepurton and tamiebadman = 31,  index is 19
fuzz for k

fuzz for drackecoslett and worthingtonwrixon = 20,  index is 18
fuzz for betteannepurton and worthingtonwrixon = 38,  index is 19
fuzz for katharinastopper and worthingtonwrixon = 42,  index is 20
fuzz for kimberleylygoe and worthingtonwrixon = 19,  index is 21
fuzz for kimlygoe and worthingtonwrixon = 24,  index is 22
fuzz for pasqualebeevers and worthingtonwrixon = 6,  index is 23
fuzz for byromvanini and worthingtonwrixon = 36,  index is 24
fuzz for karlottamacgray and worthingtonwrixon = 25,  index is 25
fuzz for paulifairclough and worthingtonwrixon = 19,  index is 26
fuzz for kirbeehartles and worthingtonwrixon = 20,  index is 27
fuzz for tammiebadman and worthingtonwrixon = 21,  index is 28
fuzz for chestonbrowne and worthingtonwrixon = 47,  index is 29
fuzz for worthingtonwrixon and worthingtonwrixon = 100,  index is 30
fuzz for suttonpower and worthingtonwrixon = 43,  index is 0
fuzz for mikeltrayton and worthingtonwrixon = 34,  index is 1
fuzz for daliabibb and worthingtonwri

fuzz for tammiebadman and hastingsmeggison = 29,  index is 28
fuzz for chestonbrowne and hastingsmeggison = 41,  index is 29
fuzz for worthingtonwrixon and hastingsmeggison = 42,  index is 30
fuzz for rayegatenby and hastingsmeggison = 30,  index is 31
fuzz for konstantinkordt and hastingsmeggison = 32,  index is 32
fuzz for ailsundovydenas and hastingsmeggison = 32,  index is 33
fuzz for ailsun and hastingsmeggison = 36,  index is 34
fuzz for dionnecockcroft and hastingsmeggison = 26,  index is 35
fuzz for hastingsmeggison and hastingsmeggison = 100,  index is 36
fuzz for suttonpower and florimatuskiewicz = 14,  index is 0
fuzz for mikeltrayton and florimatuskiewicz = 28,  index is 1
fuzz for daliabibb and florimatuskiewicz = 31,  index is 2
fuzz for myrvynbuddock and florimatuskiewicz = 20,  index is 3
fuzz for cammywindless and florimatuskiewicz = 20,  index is 4
fuzz for cameronwindles and florimatuskiewicz = 26,  index is 5
fuzz for elenoremacdonnell and florimatuskiewicz = 35,  i

fuzz for katharinastopper and madisonpiebus = 41,  index is 20
fuzz for kimberleylygoe and madisonpiebus = 22,  index is 21
fuzz for kimlygoe and madisonpiebus = 29,  index is 22
fuzz for pasqualebeevers and madisonpiebus = 36,  index is 23
fuzz for byromvanini and madisonpiebus = 42,  index is 24
fuzz for karlottamacgray and madisonpiebus = 14,  index is 25
fuzz for paulifairclough and madisonpiebus = 29,  index is 26
fuzz for kirbeehartles and madisonpiebus = 23,  index is 27
fuzz for tammiebadman and madisonpiebus = 32,  index is 28
fuzz for chestonbrowne and madisonpiebus = 31,  index is 29
fuzz for worthingtonwrixon and madisonpiebus = 27,  index is 30
fuzz for rayegatenby and madisonpiebus = 25,  index is 31
fuzz for konstantinkordt and madisonpiebus = 21,  index is 32
fuzz for ailsundovydenas and madisonpiebus = 43,  index is 33
fuzz for ailsun and madisonpiebus = 42,  index is 34
fuzz for dionnecockcroft and madisonpiebus = 36,  index is 35
fuzz for hastingsmeggison and madison

fuzz for jahvu and sarafield = 14,  index is 50
fuzz for t.j.fry and sarafield = 12,  index is 51
fuzz for t-jayfry and sarafield = 24,  index is 52
fuzz for t-jfrye and sarafield = 25,  index is 53
fuzz for toddjones and sarafield = 11,  index is 54
fuzz for billsmith and sarafield = 22,  index is 55
fuzz for sarahfield and sarafield = 95,  index is 56


In [64]:

groups

[['suttonpower',
  'suttonpower',
  'suttinpower',
  'suttonpower',
  'suttonpower',
  'suttonpower',
  'suttonpowers',
  'suttonpowers'],
 ['mikeltrayton',
  'mikeltrayton',
  'michaeltrayton',
  'miketrayton',
  'mikeltrayton',
  'mikeltrayton'],
 ['daliabibb', 'daliabibb'],
 ['myrvynbuddock',
  'mervinbuddock',
  'myrvinbuddock',
  'mervynbuddock',
  'mervinbuddock'],
 ['cammywindless', 'camwindless'],
 ['cameronwindles'],
 ['elenoremacdonnell', 'elenormcdonnell', 'elenoramacdonel'],
 ['janetbiasini', 'janetbisini'],
 ['darbysteinson', 'darbiesteiinson'],
 ['maynardskirvan', 'maynardskirvanne', 'maynardskiirvane', 'maynardskirvane'],
 ['dantewillder', 'dantewild'],
 ['renahawkshaw'],
 ['annettadelamere', 'annedelamere', 'annettedelamere'],
 ['annettedelmar'],
 ['robersgammade', 'robertgammade', 'robersgamade', 'robersgammade'],
 ['tobinrutigliano', 'tobyrutigliano'],
 ['damonsteer', 'damonsteer', 'damonsteer', 'damionsteer'],
 ['francoismandifield',
  'francmandifield',
  'francoism

In [75]:
fuzz.token_sort_ratio("zay field","say field")

89

In [71]:
firstname = "may"
lastname = "zield"
if(firstname < lastname):
    fullname = firstname+lastname
else:
    fullname = lastname+firstname
fullname


'mayzield'